In [ ]:
# standard snippets I use a lot

# for auto-reloading extensions - helpful if you're writing and testing a package
%reload_ext autoreload
%autoreload 2

# for inline plotting in python using matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

# for easier plots - also makes matplotlib plots look nicer by default
import seaborn as sns

# set up for using plotly offline without an API key - great for interactive plots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.tools as pltools
init_notebook_mode(connected=True)

# for numerical work
import pandas as pd
import numpy as np

In [ ]:
import json
import juno.junodb as junodb
import juno.junoutils as junoutils

import requests

In [ ]:
# load the database credentials from file
with open('../user_aggregation_pipeline//creds.json') as json_data:
    creds = json.load(json_data)

# set up a database with credentials
db = junodb.Database(creds)

# get the full history of interac requests
all_fulfilled = list(db._client['production']['eventCollection'].find({
        'eventAction': 'fulfilled',
        'metadata.email': {'$ne': None}}))

In [ ]:
# get the full history of interac requests
def flag_emails(email_column):

    whitelist = pd.DataFrame(junoutils.flattenObjects(list(db._client['production']['emailWhitelistCollection'].find({'level': 'ALLOWED'}))))
    blacklist = pd.DataFrame(junoutils.flattenObjects(list(db._client['production']['emailBlacklistCollection'].find({'level': 'BLOCKED'}))))
    
    def check_email(email):
        if email in whitelist.email.values:
            return 'whitelist'
        elif email in blacklist.email.values:
            return 'blacklist'
        elif 'fingerfoodstudios' in email or 'test' in email or 'einstein.exchange' in email:
            return 'testing'
        else:
            return 'normal'
        
    return email_column.apply(check_email)

In [ ]:
# get the full history of interac requests
all_trade = list(db._client['production']['eventCollection'].find({
        'eventCategory': 'trade',
        'metadata.tradesResponse': 'Accepted'}))

In [ ]:
trades_df = pd.DataFrame(junoutils.flattenObjects(all_trade)).sort_values(by='created', ascending=False)

display(trades_df.head())

trades_df['email_flag'] = flag_emails(trades_df['metadata.email'])
trades_df = trades_df[((trades_df['metadata.instrument'] == 'BTCUSD') & (pd.to_numeric(trades_df['metadata.amount']) > 200)) == False]


trades_df = trades_df[trades_df.email_flag == 'normal']

trades_df = trades_df[['created','eventAction','eventCategory','eventLabel','metadata.amount','metadata.instrument','value','metadata.lastTradedPx','metadata.email']]
trades_df['metadata.instrument'] = trades_df['metadata.instrument'].str.replace('USD','')
trades_df['date'] = trades_df['created'].apply(lambda x: str(x)[0:10])
trades_df['month'] = trades_df['created'].apply(lambda x: str(x)[0:8])+'01'
trades_df['estimated_value'] = pd.to_numeric(trades_df['metadata.lastTradedPx'])*pd.to_numeric(trades_df['metadata.amount'])
trades_df['metadata.amount'] = pd.to_numeric(trades_df['metadata.amount'])


trade_summary_day = trades_df.groupby(['date','metadata.instrument','eventLabel'], as_index=False)[['metadata.amount','estimated_value']].agg(['mean','sum','count']).reset_index()
trade_summary_day.columns = [col[0] if col[1] == '' else col[0]+"_"+col[1] for col in trade_summary_day.columns]

trade_summary_day = trade_summary_day.rename(columns={'metadata.instrument': "currency",
                                                            'eventLabel': 'transaction_type',
                                                            'metadata.amount_mean': 'average_trade_amount',
                                                            'metadata.amount_sum': 'total_trade_amount',
                                                            'estimated_value_mean': 'average_trade_value',
                                                            'estimated_value_sum': 'total_trade_value',
                                                            'estimated_value_count': 'number_of_trades'})

bitcoin_traded = trade_summary_day[trade_summary_day.currency == 'BTC']
bitcoin_traded['date'] = pd.to_datetime(bitcoin_traded['date'])
bitcoin_traded

In [ ]:
# flatten objects into a pandas dataframe
deposits = junoutils.flattenObjects(all_fulfilled)

# subset the columns
cols_to_use = ['created','eventCategory','eventAction','eventLabel','metadata.amount','metadata.email','metadata.rate','metadata.cents','value']

deposits = deposits[cols_to_use]
deposits.loc[deposits.eventLabel.str.lower() == 'bitcoin', 'eventLabel'] = 'BTC'
deposits['metadata.amount'] = pd.to_numeric(deposits['metadata.amount'])

deposits['date'] = deposits['created'].apply(lambda x: str(x)[0:10])
deposits['month'] = deposits['created'].apply(lambda x: str(x)[0:8])+'01'

deposits['email_flag'] = flag_emails(deposits['metadata.email'])

deposits = deposits[((deposits.eventLabel == 'BTC') & (pd.to_numeric(deposits['metadata.amount']) > 1000)) == False]

deposits.head()

In [ ]:
summary_by_day = deposits.groupby(['date','eventLabel'], as_index=False)[['metadata.amount','value']].agg([np.mean,'count',np.sum]).reset_index()
summary_by_day.columns = [col[0] if col[1] == '' else col[0]+'_'+col[1] for col in summary_by_day.columns.ravel()]
summary_by_day.head()

In [ ]:
# use free alpha vantage API to get historical bitcoin price
api_key = 'JACPKQ4PTI6OVJ6T'

# api request url
request_url = 'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=USD&apikey=JACPKQ4PTI6OVJ6T'

r = requests.get(request_url)
json = r.json()

# get the time series object from the json
time_series = json['Time Series (Digital Currency Daily)']

# get out the dates and values
dates = [date for date in time_series.keys()]
values = [time_series[key] for key in time_series.keys()]
currency_prices = pd.DataFrame(values)
currency_prices['date'] = dates
currency_prices['month'] = currency_prices['date'].str.slice(0,8)+"01"


join_deposits = currency_prices.set_index('date').join(summary_by_day[summary_by_day.eventLabel == 'BTC'].set_index('date')).dropna()
join_deposits = join_deposits[['4a. close (USD)','5. volume','metadata.amount_mean','metadata.amount_count','metadata.amount_sum','value_mean','value_sum']]
join_deposits.columns = ['close (USD)', 'volume', 'average_amount','number_of_deposits','total_amount','average_value','total_value']
join_deposits['total_value'] = join_deposits.total_value/100
join_deposits['average_value'] = join_deposits.average_value/100
join_deposits = join_deposits.reset_index()
join_deposits['date'] = pd.to_datetime(join_deposits.date)

temp = join_deposits[['date']]
temp['currency'] = 'BTC'
temp1 = temp
temp2 = temp

temp1['transaction_type'] = 'sell'
temp2['transaction_type'] = 'buy'
temp = pd.concat((temp1,temp2))
display(temp)

bitcoin_traded_daily = temp.set_index(['date','transaction_type','currency']).join(bitcoin_traded.set_index(['date','transaction_type','currency'])).reset_index()

display(bitcoin_traded_daily)

bitcoin_closing_price = go.Scatter(
    x=join_deposits['date'],
    y=join_deposits['close (USD)'],
    name='BTC close (USD)'
)
bitcoin_closing_price_rolling_mean = go.Scatter(
    x=join_deposits['date'],
    y=pd.rolling_mean(join_deposits['close (USD)'], 7),
    name='BTC close (USD) Rolling Mean'
)
number_of_deposits = go.Scatter(
    x=join_deposits['date'],
    y=join_deposits['number_of_deposits'],
    name='Number of Deposits',
    xaxis='x2',
    yaxis='y2'
)
average_amount = go.Scatter(
    
    x=join_deposits['date'],
    y=join_deposits['average_amount'],
    name='Average Bitcoin Amount',

    xaxis='x3',
    yaxis='y3'
)
total_amount = go.Scatter(
    
    x=join_deposits['date'],
    y=join_deposits['total_amount'],
    name='Total BTC',

    xaxis='x4',
    yaxis='y4'
)
average_value = go.Scatter(
    
    x=join_deposits['date'],
    y=join_deposits['average_value'],
    name='Average Value of Deposits',

    xaxis='x5',
    yaxis='y5'
)
total_value = go.Scatter(
    
    x=join_deposits['date'],
    y=join_deposits['total_value'],
    name='Total Value of Deposits',

    xaxis='x6',
    yaxis='y6'
)

total_sells = go.Scatter(
    
    x=bitcoin_traded_daily['date'][bitcoin_traded_daily.transaction_type == 'sell'],
    y=bitcoin_traded_daily['number_of_trades'][bitcoin_traded_daily.transaction_type == 'sell'],
    name='Number of Sell Trades',

    xaxis='x7',
    yaxis='y7'
)

total_buys = go.Scatter(
    
    x=bitcoin_traded_daily['date'][bitcoin_traded_daily.transaction_type == 'buy'],
    y=bitcoin_traded_daily['number_of_trades'][bitcoin_traded_daily.transaction_type == 'buy'],
    name='Number of Buy Trades',

    xaxis='x8',
    yaxis='y8'
)

buy_average_amount = go.Scatter(
    
    x=bitcoin_traded_daily['date'][bitcoin_traded_daily.transaction_type == 'buy'],
    y=bitcoin_traded_daily['average_trade_amount'][bitcoin_traded_daily.transaction_type == 'buy'],
    name='Average Amount of Buy Trades',

    xaxis='x8',
    yaxis='y8'
)

sell_average_amount = go.Scatter(
    
    x=bitcoin_traded_daily['date'][bitcoin_traded_daily.transaction_type == 'sell'],
    y=bitcoin_traded_daily['average_trade_amount'][bitcoin_traded_daily.transaction_type == 'sell'],
    name='Average Amount of Sell Trades',

    xaxis='x9',
    yaxis='y9'
)

buy_total_amount = go.Scatter(
    
    x=bitcoin_traded_daily['date'][bitcoin_traded_daily.transaction_type == 'buy'],
    y=bitcoin_traded_daily['total_trade_amount'][bitcoin_traded_daily.transaction_type == 'buy'],
    name='Total Amount of Buy Trades',

    xaxis='x8',
    yaxis='y8'
)

sell_total_amount = go.Scatter(
    
    x=bitcoin_traded_daily['date'][bitcoin_traded_daily.transaction_type == 'sell'],
    y=bitcoin_traded_daily['total_trade_amount'][bitcoin_traded_daily.transaction_type == 'sell'],
    name='Total Amount of Sell Trades',

    xaxis='x9',
    yaxis='y9'
)

buy_average_value = go.Scatter(
    
    x=bitcoin_traded_daily['date'][bitcoin_traded_daily.transaction_type == 'buy'],
    y=bitcoin_traded_daily['average_trade_value'][bitcoin_traded_daily.transaction_type == 'buy'],
    name='Average Estimated Value of Buy Trades',

    xaxis='x8',
    yaxis='y8'
)

sell_average_value = go.Scatter(
    
    x=bitcoin_traded_daily['date'][bitcoin_traded_daily.transaction_type == 'sell'],
    y=bitcoin_traded_daily['average_trade_value'][bitcoin_traded_daily.transaction_type == 'sell'],
    name='Average Estimated Value of Sell Trades',

    xaxis='x9',
    yaxis='y9'
)

buy_total_value = go.Scatter(
    
    x=bitcoin_traded_daily['date'][bitcoin_traded_daily.transaction_type == 'buy'],
    y=bitcoin_traded_daily['total_trade_value'][bitcoin_traded_daily.transaction_type == 'buy'],
    name='Total Estimated Value of Buy Trades',

    xaxis='x8',
    yaxis='y8'
)

sell_total_value = go.Scatter(
    
    x=bitcoin_traded_daily['date'][bitcoin_traded_daily.transaction_type == 'sell'],
    y=bitcoin_traded_daily['total_trade_value'][bitcoin_traded_daily.transaction_type == 'sell'],
    name='Total Estimated Value of Sell Trades',

    xaxis='x9',
    yaxis='y9'
)

fig = pltools.make_subplots(rows=10, cols=1)

fig.append_trace(bitcoin_closing_price, 1, 1)
fig.append_trace(bitcoin_closing_price_rolling_mean, 1, 1)
fig.append_trace(number_of_deposits, 2, 1)
fig.append_trace(average_amount, 3, 1)
fig.append_trace(total_amount, 4, 1)
fig.append_trace(average_value, 5, 1)
fig.append_trace(total_sells, 6, 1)
fig.append_trace(total_buys, 6, 1)
fig.append_trace(buy_average_amount, 7, 1)
fig.append_trace(sell_average_amount, 7, 1)
fig.append_trace(buy_total_amount, 8, 1)
fig.append_trace(sell_total_amount, 8, 1)
fig.append_trace(buy_average_value, 9, 1)
fig.append_trace(sell_average_value, 9, 1)
fig.append_trace(buy_total_value, 10, 1)
fig.append_trace(sell_total_value, 10, 1)


fig['layout'].update(height=1000, width=1600, title='Bitcoin Price v. Credit Card and Interac Deposits by Day')


iplot(fig)

In [ ]:
trades_df[(trades_df['metadata.instrument'] == 'BTC') & (trades_df['metadata.amount'] > 10)]

In [ ]:
trades_df[trades_df['metadata.email'] == 'jessica.cuteniss@icloud.com']